# Kriging

In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

In [ ]:
import gstlearn as gl
import gstlearn.plot as gp
import matplotlib.pyplot as plt
import numpy as np
import os

Then we download the data base **dat**.

In [ ]:
fileNF = os.path.join(os.getenv('GSTLEARN_DATA'), "Scotland", "Scotland_Temperatures.NF")
dat = gl.Db.createFromNF(fileNF)

Calculate the experimental variogram **vario2dir** (in 2 directions)

In [ ]:
varioParamMulti = gl.VarioParam.createMultiple(2, 15, 15.)
vario2dir = gl.Vario(varioParamMulti, dat)
err = vario2dir.compute()

Calculate the fitted model **fitmodOK** (add the Universality Condition)

In [ ]:
fitmodOK = gl.Model()
err = fitmodOK.fit(vario2dir,types=[gl.ECov.NUGGET, gl.ECov.EXPONENTIAL, gl.ECov.GAUSSIAN])
err = fitmodOK.addDrift(gl.Drift1())
ax = gp.varmod(vario2dir, fitmodOK)

Define the Unique Neighborhood **unique.neigh**.

In [ ]:
uniqueNeigh = gl.NeighUnique.create()

Get the extension of the Data:

In [ ]:
dat.getExtremas()

Create the Target file **grid**.

In [ ]:
grid = gl.DbGrid.create(x0=[65,535],dx=[4.94, 4.96],nx=[81,137])
dbfmt = gl.DbStringFormat.createFromFlags(flag_resume=False, flag_vars=True,
                                          flag_extend=True, flag_stats=False,
                                          flag_array=False)
grid.display(dbfmt)

Perform Ordinary Kriging

In [ ]:
err = gl.kriging(dat,grid,fitmodOK,uniqueNeigh,namconv=gl.NamingConvention("OK"))

Plotting the results

In [ ]:
gp.setDefaultGeographic(dims=[8,8])
ax = grid.plot()
ax.decoration(title="Ordinary Kriging over whole Grid")

Reading the Elevation Grid

In [ ]:
fileNF = os.path.join(os.getenv('GSTLEARN_DATA'), "Scotland", "Scotland_Elevations.NF")
grid = gl.DbGrid.createFromNF(fileNF)
grid.display(dbfmt)

The output grid now contains the selection **inshore**. 
Estimation is restricted to the active cells only.

In [ ]:
err = gl.kriging(dat,grid,fitmodOK,uniqueNeigh,namconv=gl.NamingConvention("OK"))

Plotting the Estimation

In [ ]:
ax = grid.plot("OK*estim")
ax = dat.plot(name_size="January_temp")
ax.decoration(title="Estimation by Ordinary Kriging")

Plotting the Standard deviation of Estimation Error

In [ ]:
ax = grid.plot("OK*stdev")
ax = dat.plot(name_size="January_temp")
ax.decoration(title="St. dev. by Ordinary Kriging")

The Model **fitmodOK** is first duplicated into **fitmodSK**. Then the Universality Condition is deleted.

In [ ]:
fitmodSK = fitmodOK.clone()
err = fitmodSK.delDrift(0)
err = fitmodSK.setMean(20.)

Simple Kriging is performed

In [ ]:
err = gl.kriging(dat,grid,fitmodSK,uniqueNeigh, namconv=gl.NamingConvention("SK"))

Plotting the Estimation

In [ ]:
ax = grid.plot("SK*estim")
ax = dat.plot(name_size="January_temp")
ax.decoration(title="Estimation by Simple Kriging")

Plotting the Standard deviation of the Estimation error

In [ ]:
ax = grid.plot("SK*stdev")
ax = dat.plot(name_size="January_temp",flagCst=True)
ax.decoration(title="St. dev. by Simple Kriging")

Comparing Ordinary and Simple Kriging - Estimations

In [ ]:
ax = gp.correlation(grid,namex="OK*estim",namey="SK*estim", bissLine=True, bins=100, flagSameAxes=True)
ax.decoration(title="Estimation Simple vs. Ordinary", 
              xlabel="Ordinary Kriging", ylabel="Simple Kriging")

Comparing Ordinary and Simple Kriging - St. dev.

In [ ]:
ax = gp.correlation(grid,namex = "OK*stdev",namey="SK*stdev", bissLine=True, bins=100,flagSameAxes=True)
ax.decoration(title="St. dev. Simple vs. Ordinary", 
              xlabel="Ordinary Kriging", ylabel="Simple Kriging")

Cross-Validation

In [ ]:
err = gl.xvalid(dat,fitmodOK,uniqueNeigh, 
                namconv=gl.NamingConvention("Xvalid", True, True, False, gl.ELoc.UNKNOWN))

Cross-validation - Histogram of Errors

In [ ]:
ax = gp.histogram(dat,name="*esterr*",bins=30,fill="blue")
ax.decoration(xlabel="Estimation Errors",title="Cross-Validation")

Cross-validation - Histogram of Standardized Errors

In [ ]:
ax = gp.histogram(dat,name="*stderr*",bins=30,fill="blue")
ax.decoration(xlabel="Standardized Errors", title="Cross-Validation")

Cross-validation Scores

In [ ]:
print(round(np.nanmean(dat.getColumn("*esterr*")),4))
print(round(np.nanmean(np.square(dat.getColumn("*esterr*"))),4))
print(round(np.nanmean(np.square(dat.getColumn("*stderr*"))),4))

Display Errors of the Cross Validation

In [ ]:
ax = grid.plot("inshore")
ax = dat.plot(name_size="*esterr",sizmax=300)
ax.decoration(title="Cross-Validation scores")

Display Errors of the Cross Validation (in absolute value)

In [ ]:
ax = grid.plot("inshore")
ax = dat.plot(name_size="*esterr",sizmax=300,flagAbsSize=True)
ax.decoration(title="Cross-Validation scores (abs. value)")

We design a small Moving Neighborhood **small.neigh** with only 1 sample per neighborhood.

In [ ]:
smallNeigh = gl.NeighMoving.create(flag_xvalid=False, nmini=1, nmaxi=1, radius=1000000)

We perform Ordinary Kriging

In [ ]:
err = gl.kriging(dat,grid,fitmodOK,smallNeigh,namconv=gl.NamingConvention("Small"))

Graphic representation

In [ ]:
ax = grid.plot("Small*estim")
ax = dat.plot(name_size="January_temp")
ax.decoration(title="Estimation by Ordinary Kriging (Small Moving Neigh.)")

Building a reasonable Moving Neighborhood, although with a limited extension (*radius*)

In [ ]:
movingNeigh = gl.NeighMoving.create(nmini=1, nmaxi=10, radius=20)

Running the Ordinary Kriging

In [ ]:
err = gl.kriging(dat,grid,fitmodOK,movingNeigh, namconv=gl.NamingConvention("Reduced"))

Plotting the results

In [ ]:
ax = grid.plot("Reduced*estim")
ax = dat.plot(name_size="January_temp")
ax.decoration(title="Estimation by Ordinary Kriging (Reduced Moving Neigh.)")

Lots of target sites are not estimated as no sample is found within the neighborhood.

Building a reasonable Moving Neighborhood correctly tuned: 10 samples (maximum) selected in a radius of 150 around the target site.

In [ ]:
movingNeigh = gl.NeighMoving.create(nmini=1, nmaxi=10, radius=150)

Running the Ordinary Kriging

In [ ]:
err = gl.kriging(dat,grid,fitmodOK,movingNeigh, namconv=gl.NamingConvention("Moving"))

Graphic representation

In [ ]:
ax = grid.plot("Moving*estim")
ax = dat.plot(name_size="January_temp")
ax.decoration(title="Estimation by Ordinary Kriging (Moving Neigh.)")

For the standard deviation of Estimation error

In [ ]:
ax = grid.plot("Moving*stdev")
ax = dat.plot(name_size="January_temp")
ax.decoration(title="St. dev. by Ordinary Kriging (Moving Neigh.)")

Comparing Unique and Moving Neighborhoods: Estimations

In [ ]:
ax = gp.correlation(grid,namex = "OK*estim",namey="Moving*estim", bins=100, bissLine=True, flagSameAxes=True)
ax.decoration(title="Comapring Estimations", xlabel="Unique", ylabel="Moving")

Comparing Unique and Moving Neighborhoods: Standard deviations

In [ ]:
ax = gp.correlation(grid,namex = "OK*stdev",namey="Moving*stdev", bins=100, bissLine=True, flagSameAxes=True)
ax.decoration(title="Comparing St. Deviations", xlabel="Unique", ylabel="Moving")